In [ ]:
import urllib
import urllib.request as ur
import numpy
import csv
import copy

In [ ]:
BASEPATH = "C:/Users/Kyle/Assignment 1/"
EHTA = .00001

In [ ]:
def parseJSONData(fname):
    with open(fname) as f:
        content = f.readlines()
    for l in content:
        yield eval(l)

def readCSV(fname):
    with open(fname) as f:
        reader = list(csv.reader(f,delimiter = '-'))
    return reader

def getYs(JSON):
    values = []
    for i in range(len(JSON)):
        values.append(JSON[i]['rating'])
    return values

In [ ]:
def d_alpha(u1, i1, a1, alpha, n, EHTA): #times 1/n? 
    suma = 0
    count = 0
    for user in u1:
        for item in u1[user]['items']:
            count += 1
            suma += (a1 + i1[item]['itemBias'] + u1[user]['userBias'] + numpy.dot(i1[item]['gamma'], u1[user]['gamma']) - int(u1[user]['items'][item]))
    #suma = suma / count
    alpha -= (EHTA * suma)
    return alpha

def d_beta_u(u1, userdata, i1, a1, k, lam, EHTA):
    for user in u1:
        sumb = 0
        sumg= numpy.zeros(k)
        for item in u1[user]['items']:
            sumb += (a1 + i1[item]['itemBias'] + u1[user]['userBias'] + numpy.dot(i1[item]['gamma'], u1[user]['gamma']) - int(u1[user]['items'][item]))
            sumb += (lam * u1[user]['userBias'])
            sumg += (a1 + i1[item]['itemBias'] + u1[user]['userBias'] + numpy.dot(i1[item]['gamma'], u1[user]['gamma']) - int(u1[user]['items'][item])) * i1[item]['gamma']
        #sumb = sumb / len(u1[user]['items'])
        #sumg = sumg / len(u1[user]['items'])
        userdata[user]['userBias'] -= (100 * EHTA * sumb)
        userdata[user]['gamma'] -= (100 * EHTA * sumg)
    #return userdata

def d_beta_i(u1, i1, itemdata, a1, k, lam, EHTA):
    for item in i1:
        sumb = 0
        sumg = numpy.zeros(k)
        for user in i1[item]['users']:
            sumb += (a1 + u1[user]['userBias'] + i1[item]['itemBias'] + numpy.dot(u1[user]['gamma'], i1[item]['gamma']) - int(i1[item]['users'][user]))
            sumb += (lam * i1[item]['itemBias'])
            sumg += (a1 + i1[item]['itemBias'] + u1[user]['userBias'] + numpy.dot(i1[item]['gamma'], u1[user]['gamma']) - int(i1[item]['users'][user])) * u1[user]['gamma']
        #sumb = sumb / len(i1[item]['users'])
        #sumg = sumg / len(i1[item]['users'])
        itemdata[item]['itemBias'] -= (100 * EHTA * sumb)
        itemdata[item]['gamma'] -= (100 * EHTA * sumg)
    #return itemdata

In [ ]:
def update_all(userdata, itemdata, alpha, n, k, lam, eps, EHTA): #make copies before calling them all
    dif = 1
    count = 0
    fvalold = 1000000000
    fval = 100000000
    mseold = 7
    mse = getMSE(alpha, userdata, itemdata, Validation)
    
    while(fvalold > fval and mseold >= mse):
        a1 = copy.deepcopy(alpha)
        u1 = copy.deepcopy(userdata)
        i1 = copy.deepcopy(itemdata)
        if(count %10 == 0):
            mseold = mse
            mse = getMSE(alpha, userdata, itemdata, Validation)
            print("Count: ", count, "MSE = ", mse)
            print(alpha)
        alpha = d_alpha(u1, i1, a1, alpha, n, EHTA)
        d_beta_u(u1, userdata, i1, a1, k, lam, EHTA)
        d_beta_i(u1, i1, itemdata, a1, k, lam, EHTA)
        fvalold = fval
        fval = f(userdata, itemdata, alpha, lam)
        print(fval)

        count += 1

    return alpha

def f(userdata, itemdata, alpha, lam):
    sum = 0
    for user in userdata:
        for item in userdata[user]['items']:
            sum += (alpha + userdata[user]['userBias'] + itemdata[item]['itemBias'] + numpy.dot(itemdata[item]['gamma'], userdata[user]['gamma']) - int(userdata[user]['items'][item]))**2
            sum += lam * (itemdata[item]['itemBias']**2 + numpy.linalg.norm(itemdata[item]['gamma']))
        sum += lam * (userdata[user]['userBias']**2 + numpy.linalg.norm(userdata[user]['gamma']))
    return sum

In [ ]:
def getUserData(Training, k):
    userdata = {}
    gamma_u = numpy.random.rand(k)*.01
    #gamma_u = numpy.zeros(k)
    for element in Training:
        key = element["reviewerID"]
        if(key not in userdata):
            userdata[key] = {
                'userBias' : numpy.random.uniform(-.1, .1), 
                'items' : {
                    element['itemID'] : element['rating']
                },
                'gamma' : gamma_u
            }
        else:
            userdata[key]['items'][element['itemID']] = element['rating']
    return userdata

def getItemData(Training, k):
    itemdata = {}
    gamma_i = numpy.random.rand(k)*.01
    #gamma_i = numpy.zeros(k)
    for element in Training:
        key = element["itemID"]
        if(key not in itemdata):
            itemdata[key] = {
                'itemBias' : numpy.random.uniform(-.1, .1), 
                'users' : {
                    element['reviewerID'] : element['rating']
                },
                'gamma' : gamma_i
            }
        else:
            itemdata[key]['users'][element['reviewerID']] = element['rating']
    return itemdata

In [ ]:
data = list(parseJSONData(BASEPATH + "train.json"))

Training = data[:100000]
Validation = data[100000:200000]

In [ ]:
lam = 10
k = 1
eps = .0001
alpha = 4.23198
n = len(Training)

userdata = getUserData(Training, k)
itemdata = getItemData(Training, k)

In [ ]:
alpha = update_all(userdata, itemdata, alpha, n, k, lam, eps, EHTA)

In [ ]:
def getMSE(alpha, userdata, itemdata, data):
    sum = 0
    for i in range(len(data)):
        user = 0
        item = 0
        dot = 0
        if(data[i]['reviewerID'] in userdata):
            user = userdata[data[i]['reviewerID']]['userBias']
        if(data[i]['itemID'] in itemdata):
            item = itemdata[data[i]['itemID']]['itemBias']
        if(user and item):
            dot  = numpy.dot(userdata[data[i]['reviewerID']]['gamma'], itemdata[data[i]['itemID']]['gamma'])
        sum += ((alpha + user + item + dot - data[i]['rating'])**2)
    sum = sum / len(data)
    return sum

In [ ]:
print("MSE = ", getMSE(alpha, userdata, itemdata, Validation))

In [ ]:
def getPredict(alpha, userdata, itemdata, pairs):
    prediction = numpy.zeros(14000)
    for i in range(len(pairs)):
        user = 0
        item = 0
        if(pairs[i][0] in userdata):
            user = userdata[pairs[i][0]]['userBias']
        if(pairs[i][1] in itemdata):
            item = itemdata[pairs[i][1]]['itemBias']
        if(user and item):
            dot  = numpy.dot(itemdata[pairs[i][1]]['gamma'], userdata[pairs[i][0]]['gamma'])
        prediction[i] = (alpha + user + item + dot)
    return prediction

In [ ]:
pairs = readCSV(BASEPATH + "pairs_Rating.txt")

file = open(BASEPATH + "pairs_Rating_mine.txt", 'w')
file.write("userID-itemID,prediction\r")
predic = getPredict(alpha, userdata, itemdata, pairs)

for i in range(len(pairs)):
    file.write(str(pairs[i][0]) + "-" + str(pairs[i][1]) + "," + str(predic[i]) + "\r")

In [ ]:
for user in userdata:
    print(userdata[user]['userBias'])